In [ ]:
from langchain.chat_models import ChatOpenAI
from langchain.prompts import ChatPromptTemplate
from langchain.callbacks import StreamingStdOutCallbackHandler


chat = ChatOpenAI(
    model="gpt-3.5-turbo",
    temperature=0.8,
    streaming=True,
    callbacks=[StreamingStdOutCallbackHandler()],
)

haiku_prompt = ChatPromptTemplate.from_messages(
    [
        (
            "system",
            (
                "You are a poet who creates the most beautiful Haikus in the"
                " world. Write a three-line Haiku about the programming"
                " language '{language}'. Focus on its core essence,"
                " philosophy, or common usage traits."
            ),
        ),
        ("human", "Write a three-line Haiku about  {language}."),
    ]
)

haiku_chain = haiku_prompt | chat

haiku_exp_prompt = ChatPromptTemplate.from_messages(
    [
        (
            "system",
            (
                "Explain the Haiku below about the programming language"
                " {haiku}. Provide a detailed analysis of its lines and how"
                " they relate to the language's"
                " characteristics."
            ),
        ),
        ("human", "{haiku}"),
    ]
)

haiku_exp_chain = haiku_exp_prompt | chat

final_chain = {"haiku": haiku_chain} | haiku_exp_chain

final_chain.invoke({"language": "python"})

Elegant and clean,
Python slithers through code with
Graceful simplicity.This Haiku beautifully captures the essence of Python, a programming language known for its simplicity and readability.

1. "Elegant and clean," - Python code is often praised for its elegance and cleanliness. The language emphasizes readability and uses indentation to define code blocks, making it visually appealing and easy to understand.

2. "Python slithers through code with" - The use of "slithers" here is a clever metaphor, suggesting that Python moves smoothly and effortlessly through code. This reflects Python's concise syntax and the way it allows developers to express complex ideas in just a few lines of code.

3. "Graceful simplicity." - The most defining characteristic of Python is its simplicity. The language is designed to be easy to learn and use, with a minimalist approach that allows developers to focus on solving problems rather than getting caught up in complex syntax or unnecessary details.

Ov

AIMessageChunk(content='This Haiku beautifully captures the essence of Python, a programming language known for its simplicity and readability.\n\n1. "Elegant and clean," - Python code is often praised for its elegance and cleanliness. The language emphasizes readability and uses indentation to define code blocks, making it visually appealing and easy to understand.\n\n2. "Python slithers through code with" - The use of "slithers" here is a clever metaphor, suggesting that Python moves smoothly and effortlessly through code. This reflects Python\'s concise syntax and the way it allows developers to express complex ideas in just a few lines of code.\n\n3. "Graceful simplicity." - The most defining characteristic of Python is its simplicity. The language is designed to be easy to learn and use, with a minimalist approach that allows developers to focus on solving problems rather than getting caught up in complex syntax or unnecessary details.\n\nOverall, this Haiku effectively conveys th